In [1]:
import numpy as np
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
sns.set()

from bond_pricer import FixedCouponBond, ONE_PCT, ONE_BP

### Bond term sheet
* Maturity: 3 years
* Frequency : semiannual
* 3% coupon

In [2]:
coupon = 3 * ONE_PCT
principal = 100 * ONE_PCT
coupon_frequency = 2
maturity = 3

term_sheet = defaultdict(
    None,
    {
        'coupon': coupon,
        'principal': principal,
        'coupon_frequency': coupon_frequency,
        'maturity': maturity,
    }
)

bond = FixedCouponBond(
    term_sheet,
)

### First check that without credit risk and without rates volatility, a constant 3% short interest rate prices the bond close to par (the same way a 3% yield would)

In [3]:
model_params = {
    'mean_reversion_ir': 3.0,
    'mean_reversion_cd': 1.0,
    'long_term_ir': 0 * ONE_BP,
    'long_term_cd': 0 * ONE_BP,
    'vol_ir': 0 * ONE_BP,
    'vol_cd': 0 * ONE_BP,
    'corr_ir_cd': 0 * ONE_PCT,
}

sim_config = defaultdict(
    None,
    {
        'model_name': 'Vasicek',
        'init_ir': 0 * ONE_BP,
        'init_cd': 0 * ONE_BP,
        'scheme_steps': 5 * maturity,  # ~1 per week
        'n_mc_sim': 10,
        'model_params': model_params,
    }
)

bond.recovery_rate = 0.4
bond.funding_rate = 300 * ONE_BP
bond.sim_config = sim_config

In [4]:
np.random.seed(2)  # Fix random seed (for reproducibility of MC simulations)
print('Model price: {:.2%}'.format(bond.model_price))

Model price: 99.94%


### Then check that increasing the hazard rate decreases the price

In [5]:
model_params = {
    'mean_reversion_ir': 3.0,
    'mean_reversion_cd': 1.0,
    'long_term_ir': 0 * ONE_BP,
    'long_term_cd': 300 * ONE_BP,
    'vol_ir': 0 * ONE_BP,
    'vol_cd': 0 * ONE_BP,
    'corr_ir_cd': 0 * ONE_PCT,
}

sim_config = defaultdict(
    None,
    {
        'model_name': 'Vasicek',
        'init_ir': 0 * ONE_BP,
        'init_cd': 300 * ONE_BP,
        'scheme_steps': 5 * maturity,  # ~1 per week
        'n_mc_sim': 10,
        'model_params': model_params,
    }
)

bond.recovery_rate = 0.4
bond.funding_rate = 300 * ONE_BP
bond.sim_config = sim_config

In [6]:
np.random.seed(2)  # Fix random seed (for reproducibility of MC simulations)
print('Model price: {:.2%}'.format(bond.model_price))

Model price: 94.24%
